In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import GradientAccumulationScheduler
from pytorch_lightning import Trainer
import wandb
import math
sys.path.append('../..')

from LightningModules.TrackML_ACAT.training_utils import model_selector

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

from LightningModules.TrackML_ACAT.tracking_utils import eval_metrics

In [3]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        try:
            if name.endswith(".bias"):
                param.data.fill_(0)
            elif name.endswith("0.weight"):  # The first layer does not have ReLU applied on its input
                param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
            else:
                param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))
        except IndexError as E:
            continue

In [4]:
def load_from_pretrained(model, path = None, ckpt = None):
    
    if ckpt is None:
        ckpt = torch.load(path)
    else:
        pass
    state_dict = ckpt["state_dict"]
    model.load_state_dict(state_dict, strict=False)
    del state_dict
    
    return model

## Training

In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode="min",
    save_top_k=2,
    save_last=True)

In [6]:
model_name = input()
model = model_selector(model_name)
kaiming_init(model)

 5


In [ ]:
logger = WandbLogger(project="TrackML_1GeV")
accumulator = GradientAccumulationScheduler(scheduling={0: 1, 4: 2, 8: 4})
trainer = Trainer(gpus=1, max_epochs=model.hparams["max_epochs"], gradient_clip_val=0.5, logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/")
trainer.fit(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liuryan30 (exatrkx). Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                 | Params
----------------------------------------------------------------
0 | ignn_block             | InteractionGNNBlock  | 8.1 M 
1 | hgnn_block             | HierarchicalGNNBlock | 16.7 M
2 | bipartite_output_layer | Sequential           | 527 K 
----------------------------------------------------------------
25.3 M    Trainable params
0         Non-trainable params
25.3 M    Total params
101.200   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10385. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings

Training: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:229: UserWarning: You called `self.log('clusters', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/env

Validation: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:229: UserWarning: You called `self.log('track_eff', ...)` in your `validation_step` but the value needs to be flo

Validation: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda

In [6]:
training_id = input()
model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/TrackML_1GeV/{}/checkpoints/last.ckpt".format(training_id)
ckpt = torch.load(model_path)
model = model_selector(ckpt["hyper_parameters"]["model"], ckpt["hyper_parameters"])
    
logger = WandbLogger(project="TrackML_1GeV", id = training_id)
accumulator = GradientAccumulationScheduler(scheduling={0: 1, 4: 2, 8: 4})
trainer = Trainer(gpus=1, max_epochs=ckpt["hyper_parameters"]["max_epochs"], gradient_clip_val=0.5, logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/")
trainer.fit(model, ckpt_path="/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/TrackML_1GeV/{}/checkpoints/last.ckpt".format(training_id))

 1x2ddk1i


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liuryan30 (exatrkx). Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /global/cfs/cdirs/m3443/usr/ryanliu/TrackML/TrackML_1GeV/1x2ddk1i/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at /global/cfs/cdirs/m3443/usr/ryanliu/TrackML/TrackML_1GeV/1x2ddk1i/checkpoints/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [None]

  | Name       | Type                 | Params
----------------------------------------------------
0 | ignn_block | InteractionGNNBlock  | 2.0 M 
1 | hgnn_block | HierarchicalGNNBlock | 5.6 M 
----------------------------------------------------
7.7 M     Trainable params
1         Non-trainable params
7.7 M     Total params
30.602    

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

## Sweep

In [ ]:
run_name = input()

In [ ]:
with open("sweep.yaml") as f:
    sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)  

In [ ]:
sweep_configuration = {
    "name": run_name,
    "project": "TrackML_1GeV",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [ ]:
def training():
    wandb.init()
    
    model = model_selector(wandb.config["model"], wandb.config)
    kaiming_init(model)
    
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        mode="min",
        save_top_k=2,
        save_last=True)
    
    kaiming_init(model)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=model.hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "TrackML_1GeV")

# run the sweep
wandb.agent(sweep_id, function=training)

## Test

In [9]:
inference_config = {
    "majority_cut": float(input()),
    "score_cut": 0.7
}
model_path = input()
if "/" not in model_path:
    model_path = "/global/cfs/cdirs/m3443/usr/ryanliu/TrackML/TrackML_1GeV/{}/checkpoints/".format(model_path)
    model_paths = os.listdir(model_path)
    model_paths.remove("last.ckpt")
    ckpt_name = model_paths[0]
    for i in model_paths:
        if int(i.strip("epoch=").split("-")[0]) > int(ckpt_name.strip("epoch=").split("-")[0]):
            ckpt_name = i
    model_path = os.path.join(model_path, ckpt_name)

ckpt = torch.load(model_path)
sweep_configs = {**(ckpt["hyper_parameters"]), **inference_config}

model = model_selector(ckpt["hyper_parameters"]["model"], sweep_configs)
    
model = load_from_pretrained(model, ckpt = ckpt)
model.setup("test")
trainer = Trainer(gpus=1)
test_results = trainer.test(model, model.test_dataloader())[0]

 1
 1f2g0r0f


/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(
/global/homes/r/ryanliu/.conda

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          hit_eff          │            1.0            │
│          hit_pur          │            1.0            │
│         track_eff         │    0.5578115582466125     │
│         track_pur         │     0.551129937171936     │
│         val_loss          │   0.021192163228988647    │
└───────────────────────────┴───────────────────────────┘

In [10]:
print(*[test_results[i] for i in test_results], sep = '\n')

0.5578115582466125
0.551129937171936
1.0
1.0
0.021192163228988647


In [24]:
print(*[i for i in test_results], sep = '\n')

track_eff
track_pur
hit_eff
hit_pur
val_loss
